#The Spiking Neuron Model - Coding Challenge Problems (Part 4)


# Extending the Model: Synaptic Conductance

This interactive document is meant to be followed as the reader makes their way through chapter: *The Spiking Neuron Model*.  Each model presented in the chapter will have a section consisting of a step-by-step walkthrough of a simple Python implementation.  This is followed by an interface to run simulations with different parameter values to answer the Coding Challenge Problems.

For each model covered in the chapter, there is a section called **Simulations for Coding Challenge Problems.**  This is where you will find user-interface components such as value sliders for various parameters.  Use these controls to answer the questions from the text, and feel free to explore how the values affect the results.


**Content creators**: Maxwell E. Bohling, Dr. Lawrence Udeigwe

## How This Works

This Jupyter Notebook has both Content cells and Code cells.  As you make your way through the tutorial, you MUST make sure to run each code cell as you come to them.  Each code cell has a *Play* button next to it which will execute the code. Some code may be hidden (generally because it is more complex code not completely necessary to understand for the purposes of the chapter Coding Challenge Problems.)

**IMPORTANT**: You are currently viewing a copy of the original notebook.  You will find that you can edit the content of any cell.  If you accidently change a cell, such as a line of code and/or run into errors as you try to run subsequent blocks, simply refresh the page, OR go to the *Runtime* Menu and select *Restart runtime*.  It is also suggested that you go to the *Edit* menu and select *Clear all outputs.*  This will always allow you to revert the notebook to the original version (though you will have to run each code block again.)



 Execute the code block. **Initialize Setup**

In [ ]:
#@title Initialize Setup
#@markdown Import python libraries, and define helper functions (No need to understand this code, simply make sure you run this first.)
import sys
import functools as ft
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy as sc

def printError():
  message = 'Something went wrong!\n\n'  
  message = message + 'Check for the following:\n\n'
  message = message + '\t1.  Variables are set to the correct values.  Make sure to replace any instances of \'None\'.\n'
  message = message + '\t2.  All previous code blocks have been run the order they appear and output a success message.\n'
  message = message + '\t3.  No other code has been altered.\n\n'
  message = message + 'then try running the code block again.\n'
  print(message)

def printSuccess():
  message = 'Success!  Move on to the next section.'
  print(message)

def checkVoltageParameters(Vrest, Vth, Vreset, Vspike):
  # Check: IAF-Voltage-Parameters
  print('Checking Voltage Parameters...\n')
  try:
      check_Vrest = Vrest
      check_Vth = Vth
      check_Vreset = Vreset
      check_Vspike = Vspike
  except:
    printError()
  else:
    vals = [Vrest, Vth, Vreset, Vspike]
    correct_vals = [-70, -54, -80, 0]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      printSuccess()
    else:
      printError()
 
def checkNeuronProperties(Rm, Cm, tau_m):
  # Check: IAF-Neuron-Properties
  print('Checking Neuron Properties...\n')
  try:
      check_Rm = Rm
      check_Cm = Cm
      check_tau_m = tau_m
  except:
    printError()
  else:
    vals = [Rm, Cm, tau_m]
    correct_vals = [10, 1, 10]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      printSuccess()
    else:
      printError()

def checkSimulationSetup(Vrest, Vinitial, EL, t0, dt, t_final, time, Ie, t_pulse, start_current, end_current):
  # Check: IAF-Simulation-Setup
  print('Checking Simulation Setup...\n')
  try:
    check_Vrest = Vrest
    check_Vinitial = Vinitial
    check_EL = EL
    check_t0 = t0
    check_dt = dt
    check_t_final = t_final
    check_time = time
    check_Ie = Ie
    check_t_pulse = t_pulse
    check_start_current = start_current
    check_end_current = end_current
  except:
    printError()
  else:
    vals = [Vrest, Vinitial, EL, t0, dt, t_final, Ie, t_pulse, start_current, end_current]
    correct_vals = [-70, -70, -70, 0, 1, 500, 1.75, 300, 100, 400]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  True):  
      if len(time) == 500 and time[0] == 0 and time[-1] == 499:
        printSuccess()
      else:
        printError()
    else:
      printError()

def autoSetValues():
  Vrest = -70
  Vth = -54 
  Vreset = -80
  Vspike = 0
  Vinitial = Vrest
  EL = Vrest
  t0 = 0
  dt = 1
  t_final = 500
  time = range(t0, t_final, dt)
  t_pulse = 300 
  start_current = np.absolute(t_final-t_pulse)/2 
  end_current = start_current+t_pulse 
  V = [0] * len(time)
  V[0] = Vrest

try:
  check_sys = sys
except:
  printError()
else:
  modulename = 'functools'
  if modulename not in sys.modules:
    printError()
  else:
    printSuccess()